<a href="https://colab.research.google.com/github/younghwani/ScalpProject/blob/master/scalpResnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil
# Data file copy (from drive to colab kernel)
original_data_path = '/content/drive/MyDrive/Data/scalp.zip'
new_data_path = '/content'

shutil.copy(original_data_path, new_data_path)

In [ ]:
import zipfile
# Unzip
path_to_zip_file = '/content/scalp.zip'
directory_to_extract_to = '/content'

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
#check whether you are using GPU
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
# !pip install -U coremltools

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import keras
import math, os, sys
import matplotlib.pyplot as plt
# import coremltools

In [ ]:
def get_model():
    input_tensor = Input(shape=(224, 224, 3))

    # base pre-trained model
    base_model = ResNet50(input_tensor=input_tensor,weights='imagenet',include_top=False)

    for layer in base_model.layers:
        layer.trainable=False

    x = base_model.output
    x = GlobalAveragePooling2D(data_format='channels_last')(x)
    x = Dense(num_classes, activation='softmax')(x)

    updatedModel = Model(base_model.input, x)

    return  updatedModel

def compile_model(compiledModel):
    compiledModel.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])

def modelFitGenerator(fitModel):
    num_train_samples = sum([len(files) for r, d, files in os.walk(train_data_dir)])
    num_valid_samples = sum([len(files) for r, d, files in os.walk(validation_data_dir)])

    num_train_steps = math.floor(num_train_samples/batch_size)
    num_valid_steps = math.floor(num_valid_samples/batch_size)
    
    train_datagen = ImageDataGenerator(  
      rotation_range=90,      
      horizontal_flip=True,    
      vertical_flip=True,
      zoom_range=0.4)

    test_datagen = ImageDataGenerator()

    train_generator = train_datagen.flow_from_directory(
      train_data_dir,
      target_size=image_size ,
      batch_size=batch_size,
      class_mode='categorical', shuffle=True
    )

    validation_generator = test_datagen.flow_from_directory(
      validation_data_dir,
      target_size=image_size ,
      batch_size=batch_size,
      class_mode='categorical', shuffle=True
    )

    checkpoint = ModelCheckpoint("resnet50.h5", 
                                monitor='val_accuracy', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='auto', 
                                period=1)
    early = EarlyStopping(monitor='val_accuracy', 
                          min_delta=0, 
                          patience=10, 
                          verbose=1, 
                          mode='auto')

    print("start history model")
    history = fitModel.fit_generator(
      train_generator,
      steps_per_epoch=num_train_steps,
      epochs=nb_epoch,
      validation_data=validation_generator,
      validation_steps=num_valid_steps,
      callbacks=[checkpoint,early])

# def saveCoreMLModel(kerasModel):
#     coreml_model = coremltools.converters.keras.convert(kerasModel,
#                                                     input_names=['input'],
#                                                     output_names=['probs'],
#                                                     image_input_names='input',
#                                                     predicted_feature_name='predictedMoney',
#                                                     class_labels = 'drive/Resnet/labels.txt')
#     coreml_model.save('resnet50custom.mlmodel') 
#     print('CoreML model saved')
    
def main():
    model = get_model()
    compile_model(model)
    modelFitGenerator(model)
    # saveCoreMLModel(model)
    model.save('scalpResnetModel')

In [ ]:
if __name__ == '__main__':
    # constants
    image_size = (224, 224)
    train_data_dir = '/content/scalp/Training'
    validation_data_dir = '/content/scalp/Validation'
    nb_epoch = 100
    batch_size = 16
    num_classes = 7
    main()

In [ ]:
# Model BackUp
h5Path = '/content/resnet50.h5'
destination = '/content/drive/MyDrive'

shutil.copy(h5Path, destination)

In [ ]:
from distutils.dir_util import copy_tree

modelPath = '/content/scalpResnetModel'
destination = '/content/drive/MyDrive/scalpResnetModel'

copy_tree(modelPath, destination)

In [ ]:
# from google.colab import files
# files.download('resnet50custom.mlmodel') 

- colab kernel

~~~
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button").click() 
}setInterval(ClickConnect, 1800000)
~~~